In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

https://medium.com/@somilshah112/how-to-find-duplicate-or-similar-images-quickly-with-python-2d636af9452f


In [ ]:
df=pd.read_csv('../input/dataset-new/train_new.csv')

In [ ]:
df

In [ ]:
#df["img"] = df["Id"].apply(lambda x:x+ ".jpg")

In [ ]:
df=df[['img', 'Animals']]

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#training, validation = train_test_split(df_sum, test_size = 0.25, random_state=17)

In [ ]:
X_train=df['img']
#X_val=validation['img']
y_train=df['Animals']
#y_val=validation['Pawpularity']

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, dataframe, data_path, transform=None, target_transform=None):
        self.dataframe=dataframe
        self.data_path=data_path
        self.transform=transform
        self.target_transform=target_transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_name=self.dataframe['img'].iloc[idx]
        img=Image.open(os.path.join(self.data_path, img_name))
        img=self.transform(img)
        label=self.dataframe['Animals'].iloc[idx]
        return img, label

In [ ]:
image_size = 224 

transform = transforms.Compose([
    transforms.Resize(255), 
    transforms.CenterCrop(image_size),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])


In [ ]:
dataset_train = PetfinderDataset(df, '../input/kitty-doggy/cats', transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=32)
#dataset_val=PetfinderDataset(validation, TRAIN_PATH, transform=transform_val)
#dataloader_val=DataLoader(dataset_val, batch_size=64)

In [ ]:
from torchvision.utils import make_grid

for images, labels in dataloader_train:
    
    fig, ax = plt.subplots(figsize = (10, 10))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(images, 4).permute(1,2,0))
    print(labels)
    break

In [ ]:
pip install pretrainedmodels

In [ ]:
from torchvision import models
import pretrainedmodels


In [ ]:
resnet = models.resnet50(pretrained=True)
#resnet=resnet(include_top=False, weights='imagenet')

In [ ]:

num_ftrs = resnet.fc.in_features
print(num_ftrs)
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
#resnet.fc = nn.Linear(num_ftrs, 2)

resnet.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.Dropout(0.6),
    nn.ReLU(inplace=True),
    #nn.Dropout(0.25),
    nn.Linear(256, 2),
    nn.Sigmoid()
)


In [ ]:
resnet

In [ ]:
#CUDA_LAUNCH_BLOCKING=1.

In [ ]:
#CUDA_LAUNCH_BLOCKING=1
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    resnet.cuda()   
        

In [ ]:
import torch.optim as optim

# specify loss function
criterion = torch.nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.Adam(resnet.parameters(), lr = 0.000001)

In [ ]:
def accuracy(preds, trues):
    ### Converting preds to 0 or 1

    preds = [0 if preds[i][0]>=preds[i][1] else 1 for i in range(len(preds))]
    ### Calculating accuracy by comparing predictions with true labels
    acc = [1 if preds[i] == trues[i] else 0 for i in range(len(preds))]
    ### Summing over all correct predictions
    acc = np.sum(acc) / len(preds)
    #print(acc)
    return (acc * 100)
    #return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
import sklearn.metrics

In [ ]:
def other_metrics(preds,trues):
    #print(preds)

    y_pred = [0 if preds[i][0]>=preds[i][1] else 1 for i in range(len(preds))]

    trues= trues.to('cpu').numpy()
    
    
    r=sklearn.metrics.confusion_matrix(trues, y_pred)
    precision = sklearn.metrics.precision_score(trues, y_pred)
    recall = sklearn.metrics.recall_score(trues, y_pred)
    
    return r, precision, recall 

In [ ]:
n_epochs = 6
train_loss_min=np.Inf
train_loss_list=[]

for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    valid_loss = 0.0
    
    resnet.train()
    for data, target in dataloader_train:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = resnet(data)
        #print(output)
        loss = criterion(output, target)
        acc_train=accuracy(output, target)
        matrix_tr, precision_tr, recall_tr=other_metrics(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
        
    resnet.eval()
    
    train_loss = train_loss/len(dataloader_train.dataset)
    train_loss_list.append(train_loss)

    
    print('Epoch: {} \tTraining Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(
        epoch, train_loss, acc_train))
    
    print('Train metrics.\n Confusion Matrix:{} \t Precision: {}\t Recall {}'.format( matrix_tr, precision_tr, recall_tr))
    #print('Validation metrics.\n Confusion Matrix:{} \t Precision: {}\t Recall {}'.format( matrix_val, precision_val, recall_val))
    
    if train_loss <= train_loss_min:
        print('Train loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        train_loss_min,
        train_loss))
        torch.save(resnet.state_dict(), 'model_cifar.pt')
        train_loss_min = train_loss
    

In [ ]:
print(train_loss_list)
#print(val_loss_list)

In [ ]:
def plot_losses(train_losses):
    #train_losses = [x.get('train_loss') for x in history]
    #val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    #plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training'])
    plt.title('Loss vs. No. of epochs');

In [ ]:
plot_losses(train_loss_list)

In [ ]:
def predict_test(model, filename='../input/something/smth'):
    test_dir = '../input/something/smth'
    test_df = pd.read_csv('../input/dataset-new/test_new.csv')
    result = []
    predictor = nn.Sigmoid()
    for image_id in test_df['img']:
        img_path = os.path.join(test_dir, image_id)
        image = Image.open(img_path) # read_image
        transformed = transform(image)
        transformed = transformed.reshape(1, transformed.shape[0], transformed.shape[1], transformed.shape[2]).cuda()
        # print(transformed.shape)
        
        output = model(transformed)
        pred = predictor(output)
        result.append(pred)
        
    result_df = pd.DataFrame({'Id': test_df['img'], 'Animal': result})
    result_df.to_csv( filename + '.csv', sep='\t')

In [ ]:
predict_test(resnet, filename = '../input/something/smth')